### IMPORT OPTIONS

In [27]:
from pathlib import Path
import json
from elasticsearch import Elasticsearch

In [5]:
#Elasticsearch
print ("Connecting to elastic search")
# Password for the 'elastic' user generated by Elasticsearch
ELASTIC_PASSWORD = "DX6i+8n=85jVn9jVL0Ze"

# Create the client instance
es = Elasticsearch(
    "https://localhost:9200",
    ca_certs="/Users/balazs/elasticsearch-8.6.2/config/certs/http_ca.crt",
    basic_auth=("elastic", ELASTIC_PASSWORD)
)
print ("done")

# Successful response!
es.info()

Connecting to elastic search
done


ObjectApiResponse({'name': 'Balazss-MacBook-Pro.local', 'cluster_name': 'elasticsearch', 'cluster_uuid': 'LVjKO5l4RlGe8MMJe_Rklg', 'version': {'number': '8.6.2', 'build_flavor': 'default', 'build_type': 'tar', 'build_hash': '2d58d0f136141f03239816a4e360a8d17b6d8f29', 'build_date': '2023-02-13T09:35:20.314882762Z', 'build_snapshot': False, 'lucene_version': '9.4.2', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [11]:
# Set the file path of the input JSON file
path = "/Users/balazs/Desktop/args-me-1.0-cleaned.json"

# Print a message to the console indicating which file is being read
print ("Reading from {}".format(path))

# Open the file specified by the file path using the 'open' function, and read the first line of the file
with open(path, 'r') as fr:
    data = fr.readline()
    
# Parse the JSON data from the first line of the file using the 'json' module
data_json = json.loads(data)

Reading from /Users/balazs/Desktop/args-me-1.0-cleaned.json


In [18]:
# Print the value of the first 'argument' key in the 'arguments' list of the JSON object
print (data_json['arguments'][0])

{'premises': [{'text': 'My opponent forfeited every round. None of my arguments were answered. I don’t like the idea of winning by default, but here we are.Tule: it’s good for students to get involved and address big issues like teen pregnancy. You need to be able to answer arguments like mine and not simply prepare for an abstinence-only type of response. You should also be aware that, in the U.S., condoms may be sold to minors in ANY state. A retailer who says it is illegal to sell you them is, frankly, wrong.', 'stance': 'CON'}], 'context': {'sourceId': 'c67482ba-2019-04-18T13:32:05Z', 'previousArgumentInSourceId': '', 'acquisitionTime': '2019-04-18T13:32:05Z', 'discussionTitle': 'Contraceptive Forms for High School Students', 'sourceTitle': 'Debate Argument: Contraceptive Forms for High School Students | Debate.org', 'sourceUrl': 'https://www.debate.org/debates/Contraceptive-Forms-for-High-School-Students/1/', 'nextArgumentInSourceId': 'c67482ba-2019-04-18T13:32:05Z-00001-000'}, 'i

'\nlines = []\ni = 0\nwhile True:\n    try:\n        lines.append(next(d))\n        i=i+1\n        if i%1001 == 0:\n            print("Lines Read: ", i)\n            break\n    except Exception as e:\n        print(e)\n        break\nprint ("data loaded")\n'

In [25]:
# Set the name of the Elasticsearch index to create
index_name = "text-preprocessed"

# Create an Elasticsearch index with the specified name
es.indices.create(index=index_name, ignore=[400]) 

# Print a message to the console indicating that indexing is starting
print ("start indexing")

# Iterate over each argument in the JSON data
for i, row in enumerate(data_json['arguments']):
        # Print a message to the console every 10,000 arguments processed
    if (i+1)%10000 == 0:
        print("Lines Indexed: ", i)
        #break

    # Index the argument in Elasticsearch using the 'es.index' method
    # The argument's ID is used as the Elasticsearch document ID
    # The 'text' field of the argument's first 'premises' dictionary is used as the Elasticsearch document body
    response = es.index(
        index = index_name, 
        id = row['id'],
        body = {"text" : row["premises"][0]['text']}
    )

/var/folders/pj/99_398257tg2j0b00mlg8hqm0000gn/T/ipykernel_4000/1470033534.py:2: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.create(index=index_name, ignore=[400])
/var/folders/pj/99_398257tg2j0b00mlg8hqm0000gn/T/ipykernel_4000/1470033534.py:9: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  response = es.index(


start indexing
Lines Indexed:  9999
Lines Indexed:  19999
Lines Indexed:  29999
Lines Indexed:  39999
Lines Indexed:  49999
Lines Indexed:  59999
Lines Indexed:  69999
Lines Indexed:  79999
Lines Indexed:  89999
Lines Indexed:  99999
Lines Indexed:  109999
Lines Indexed:  119999
Lines Indexed:  129999
Lines Indexed:  139999
Lines Indexed:  149999
Lines Indexed:  159999
Lines Indexed:  169999
Lines Indexed:  179999
Lines Indexed:  189999
Lines Indexed:  199999
Lines Indexed:  209999
Lines Indexed:  219999
Lines Indexed:  229999
Lines Indexed:  239999
Lines Indexed:  249999
Lines Indexed:  259999
Lines Indexed:  269999
Lines Indexed:  279999
Lines Indexed:  289999
Lines Indexed:  299999
Lines Indexed:  309999
Lines Indexed:  319999
Lines Indexed:  329999
Lines Indexed:  339999
Lines Indexed:  349999
Lines Indexed:  359999
Lines Indexed:  369999
Lines Indexed:  379999


In [30]:
# Test
# Set the name of the Elasticsearch index to search
index_name = "text-preprocessed"

# Set the search query string to "contraception"
s = "contraception"

# Execute a search query on Elasticsearch using the 'es.search' method
# The query searches the 'text' field for the search query string
# The search results are limited to 25 hits
res = es.search(index = index_name, body={"query": {"match": {"text": s} } }, size = 100)['hits']['hits']


# Print the search results to the console
print("Elasticsearch test results: ", res)

Elasticsearch test results:  [{'_index': 'text-preprocessed', '_id': 'b79cf889-2019-04-15T20:23:02Z-00012-000', '_score': 11.866384, '_ignored': ['text.keyword'], '_source': {'text': 'More casual sex with barrier contraception is preferable to the current amount without contraception. The amount of consensual sex is not going to change no matter what the church teaches. As long as the use of barrier contraception was promoted along with this promotion of casual sex, it would be a huge net reduction in the cases of contraction of HIV. Therefore, condoning the use of barrier contraception would be the more responsible stand to take on the part of the Catholic Church.'}}, {'_index': 'text-preprocessed', '_id': 'd0b50e27-2019-04-19T12:46:41Z-00012-000', '_score': 11.841434, '_ignored': ['text.keyword'], '_source': {'text': 'Women may not have a choice, even if contraception allows them the option of reproductive control. In many developing nations, there can be a cultural preference for so

/var/folders/pj/99_398257tg2j0b00mlg8hqm0000gn/T/ipykernel_4000/3124720746.py:11: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  res = es.search(index = index_name, body={"query": {"match": {"text": s} } }, size = 100)['hits']['hits']
